# Fine tuning a BERT model using a labeled Amazon Electronics review dataset

In this fine tuning, I used an already labeled dataset in Kaggle. https://www.kaggle.com/datasets/jimtsiobikas/amazon-reviews-2018-electronics 
This dataset contains the reviewText and all these texts are classified as POSITIVE, NEGATIVE or NEUTRAL.
This dataset have two csv files and here I use the 60k csv file.
Due to computational power constraints, here I chose a random sample of 1.5k data samples with 0.5k reviews from each target class.

# Hyper parameter tuning

### Preparing the dataset to fine tune 

In [10]:
import pandas as pd

df = pd.read_csv('/kaggle/input/amazon-reviews-2018-electronics/labeled_electronics_dataset_60k.csv')
df.head()

,overall,vote,verified,reviewTime,reviewText,summary,Label
0,5,0,True,2013-11-14,Have had this approx 6 months now - is much fa...,Great NAS Unit,POSITIVE
1,5,0,True,2013-04-23,I bought my first camera bag with my camera (a...,Great Bag,POSITIVE
2,4,0,True,2017-05-31,I use this case for a Panasonic Lumix ZS50 cam...,Decent Case for a Good Price -- With Some Caveats,POSITIVE
3,1,0,True,2014-11-25,Made no difference to my Sony camcorder.,One Star,NEGATIVE
4,5,0,True,2015-02-21,Raspbmc users will love this!,1424476800,POSITIVE


In [11]:
cols = ['reviewText','Label']
df = df[cols]
df = df.rename(columns = {'reviewText':'text'})
df.head()

,text,Label
0,Have had this approx 6 months now - is much fa...,POSITIVE
1,I bought my first camera bag with my camera (a...,POSITIVE
2,I use this case for a Panasonic Lumix ZS50 cam...,POSITIVE
3,Made no difference to my Sony camcorder.,NEGATIVE
4,Raspbmc users will love this!,POSITIVE


In [12]:
# Using only 1500 records from the dataset, containing 500 reviews from each sentiment
df_selected = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(min(len(x), 500)))

# Reset index to clean up the dataframe
df_selected = df_selected.reset_index(drop=True)

# Converting NEGATIVE, NEUTRAL and POSITIVE labels to 0, 1 and 2 respectively.
df_selected['Label'] = df_selected['Label'].map({'NEGATIVE': 0, 'NEUTRAL': 1, 'POSITIVE': 2})

df_selected

/tmp/ipykernel_34/3940072638.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_selected = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(min(len(x), 500)))


,text,Label
0,Does not deliver.,0
1,This unit would not work with chromecast. It ...,0
2,The problem is not precisely the discs themsel...,0
3,After hearing and listening to reviews and vid...,0
4,Failed after 10 months,0
...,...,...
1495,Good Simple Speakers At A Good Price,2
1496,Small and Easy to Use,2
1497,Fits perfect,2
1498,Very nice range - 24-100mm. Like the clutch fo...,2


### Saving the prepared dataset

In [13]:
df_selected.to_csv('/kaggle/working/selected.csv')

### Loading the model, data and tuning hyper-parameters

In [8]:
import optuna
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

def objective(trial):
    # Define the hyperparameters to tune
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical('batch_size', [4, 8, 16])
    num_train_epochs = trial.suggest_int('num_train_epochs', 2, 5)  # Reduced max epochs
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)
    gradient_accumulation_steps = trial.suggest_int('gradient_accumulation_steps', 1, 4)

    model_name = 'bert-base-uncased'
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    dataset_path = '/kaggle/working/selected.csv'
    # Load the CSV file as a dataset
    dataset = load_dataset('csv', data_files=dataset_path)
    
    # Since the dataset is loaded with a single key, we access it with 'train'
    dataset = dataset['train']

    # Split the dataset into train and test sets
    split_dataset = dataset.train_test_split(test_size=0.2)
    
    # Access the splits
    train_dataset = split_dataset['train']
    eval_dataset = split_dataset['test']

    def preprocess_function(examples):
        inputs = tokenizer(examples['text'], truncation=True, padding='max_length')
        inputs['labels'] = examples['Label']
        return inputs
    
    train_dataset = train_dataset.map(preprocess_function, batched=True)
    eval_dataset = eval_dataset.map(preprocess_function, batched=True)
    
    # Set format for PyTorch
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    
    args = TrainingArguments(
        output_dir='./results',
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        save_total_limit=1,
        metric_for_best_model='accuracy',
        load_best_model_at_end=True,
        fp16=True,  # Enable mixed precision training
        gradient_accumulation_steps=gradient_accumulation_steps,
    )
    
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )
    
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result['eval_accuracy']

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)


[I 2024-07-09 04:10:41,622] A new study created in memory with name: no-name-d5044e11-0bfa-4954-9ab7-7040de9b14f0
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

[I 2024-07-09 05:41:31,123] Trial 18 finished with value: 0.7133333333333334 and parameters: {'learning_rate': 2.5033923850550347e-05, 'batch_size': 4, 'num_train_epochs': 4, 'weight_decay': 0.00033485598494455913, 'gradient_accumulation_steps': 1}. Best is trial 11 with value: 0.72.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.788092,0.666667,0.691792,0.663634,0.665994
2,No log,0.724573,0.696667,0.707384,0.693720,0.697521
3,No log,0.765732,0.703333,0.712283,0.701686,0.704905


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


[I 2024-07-09 05:45:40,164] Trial 19 finished with value: 0.7033333333333334 and parameters: {'learning_rate': 3.816459268604907e-05, 'batch_size': 4, 'num_train_epochs': 3, 'weight_decay': 0.0025262339383209406, 'gradient_accumulation_steps': 2}. Best is trial 11 with value: 0.72.


### Checking for the best set of hyperparameters

In [9]:
print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:
  Accuracy: 0.72
  Params: 
    learning_rate: 2.0826976349749076e-05
    batch_size: 4
    num_train_epochs: 5
    weight_decay: 1.6916972103647168e-06
    gradient_accumulation_steps: 1


### Training the model with selected best set of hyperparameters

In [14]:
best_params = {
    'learning_rate' : 2.0826976349749076e-05,
    'batch_size' : 4,
    'num_train_epochs' : 5,
    'weight_decay' : 1.6916972103647168e-06,
    'gradient_accumulation_steps' : 1
}
model_name = 'bert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(model_name)


dataset_path = '/kaggle/working/selected.csv'
# Load the CSV file as a dataset
dataset = load_dataset('csv', data_files=dataset_path)
    
# Since the dataset is loaded with a single key, we access it with 'train'
dataset = dataset['train']

# Split the dataset into train and test sets
split_dataset = dataset.train_test_split(test_size=0.2)
    
# Access the splits
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

def preprocess_function(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding='max_length')
    inputs['labels'] = examples['Label']
    return inputs
    
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)
    
# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_params['learning_rate'],
    per_device_train_batch_size=best_params['batch_size'],
    per_device_eval_batch_size=best_params['batch_size'],
    num_train_epochs=best_params['num_train_epochs'],
    weight_decay=best_params['weight_decay'],
    save_total_limit=1,
    metric_for_best_model='accuracy',
    load_best_model_at_end=True,
)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.750022,0.680000,0.668299,0.665951,0.655259
2,No log,0.670335,0.743333,0.736191,0.739560,0.734988
3,No log,0.770269,0.730000,0.734226,0.731500,0.730262
4,0.614700,0.920943,0.733333,0.739932,0.736341,0.734422
5,0.614700,1.016352,0.730000,0.729584,0.728029,0.728580


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=750, training_loss=0.4631092681884766, metrics={'train_runtime': 458.9921, 'train_samples_per_second': 13.072, 'train_steps_per_second': 1.634, 'total_flos': 1578680506368000.0, 'train_loss': 0.4631092681884766, 'epoch': 5.0})

### Fine-tuned model evaluation

In [15]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.670335054397583,
 'eval_accuracy': 0.7433333333333333,
 'eval_precision': 0.7361908666877257,
 'eval_recall': 0.7395597618539339,
 'eval_f1': 0.7349884325429757,
 'eval_runtime': 6.571,
 'eval_samples_per_second': 45.655,
 'eval_steps_per_second': 5.783,
 'epoch': 5.0}

In [16]:
trainer.save_model('/kaggle/working/models/bert-finetuned')

### Saving the fine-tuned model

In [18]:
model_save_path = './my_pretrained_model'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('./my_pretrained_model/tokenizer_config.json',
 './my_pretrained_model/special_tokens_map.json',
 './my_pretrained_model/vocab.txt',
 './my_pretrained_model/added_tokens.json',
 './my_pretrained_model/tokenizer.json')

In [19]:
import shutil
shutil.make_archive('/kaggle/working/my_pretrained_model', 'zip', '/kaggle/working/my_pretrained_model')

'/kaggle/working/my_pretrained_model.zip'